In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from modules.utils.data_processing import get_match_label

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

### Create database connection

In [2]:
database_path = "datasets/database.sqlite"
conn = sqlite3.connect(database_path)

### Read data from database

In [3]:
match_data_sql = '''select * from Match '''
team_attributes_sql = '''select * from Team_Attributes'''
player_attributes_sql = '''select player_api_id, overall_rating from Player_Attributes'''
# Country, League and Team tables don't seem to have any value for us, country_id is equivalent to country_name 
# for the neural net, as well as league_id/league name, team_id/team name
# TODO Player table may be beneficial due to some features (birthyear, weight, height), might want to add them later

match_data = pd.read_sql(match_data_sql, conn)
team_attributes = pd.read_sql(team_attributes_sql, conn)
player_attributes = pd.read_sql(player_attributes_sql, conn)

### Create labels

In [4]:
labels = match_data.apply(get_match_label, axis=1)
labels = labels.drop("match_api_id", axis=1)

match_data["label"] = labels

### Clean up some data and create new features

In [5]:
# make season numerical
match_data['season'] = match_data["season"].apply(lambda x: int(x.split("/")[0]))

# create new 'month' field
match_data['month'] = match_data["date"].apply(lambda x: int(x.split("-")[1]))

# TODO create fields which represent win/loss/draw percentage for each team (over the last x games/years)


In [6]:
def find_win_percentage(team_id):
    total_matches = match_data.loc[(match_data['home_team_api_id'] == team_id)].shape[0]
    total_matches += match_data.loc[(match_data['away_team_api_id'] == team_id)].shape[0]
    
    if total_matches == 0:
        return 0
    
    home_wins = match_data.loc[(match_data['label'] == 0) & (match_data['home_team_api_id'] == team_id)].shape[0]
    away_wins = match_data.loc[(match_data['label'] == 2) & (match_data['away_team_api_id'] == team_id)].shape[0]

    home_losses = match_data.loc[(match_data['label'] == 2) & (match_data['away_team_api_id'] == team_id)].shape[0]
    away_losses = match_data.loc[(match_data['label'] == 0) & (match_data['home_team_api_id'] == team_id)].shape[0]

    #draws = match_data.loc[(match_data['label'] == 1) & (match_data['home_team_api_id'] == team_id)].shape[0]
    #draws += match_data.loc[(match_data['label'] == 1) & (match_data['away_team_api_id'] == team_id)].shape[0]
    
    win_percentage = (home_wins + away_wins) / total_matches
    return win_percentage
    
team_attributes['win_percentage'] = team_attributes.apply(lambda row: find_win_percentage(row.team_api_id), axis = 1)

In [7]:
def find_win_percentage_against_opponent(team_1_id, team_2_id):
    total_matches = match_data.loc[(match_data['home_team_api_id'] == team_1_id) & (match_data['away_team_api_id'] == team_2_id)].shape[0]
    total_matches += match_data.loc[(match_data['home_team_api_id'] == team_2_id) & (match_data['away_team_api_id'] == team_1_id)].shape[0]
    
    if total_matches == 0:
        return 0
    
    team_1_wins = match_data.loc[(match_data['label'] == 0) & (match_data['home_team_api_id'] == team_1_id) & (match_data['away_team_api_id'] == team_2_id)].shape[0]
    team_1_wins += match_data.loc[(match_data['label'] == 2) & (match_data['home_team_api_id'] == team_2_id) & (match_data['away_team_api_id'] == team_1_id)].shape[0]
    
    win_percentage = team_1_wins/total_matches
    return win_percentage

match_data['home_team_win_percentage_against_away'] = match_data.apply(lambda row: find_win_percentage_against_opponent(row.home_team_api_id, row.away_team_api_id), axis = 1)
match_data['away_team_win_percentage_against_home'] = match_data.apply(lambda row: find_win_percentage_against_opponent(row.away_team_api_id, row.home_team_api_id), axis = 1)

In [8]:
def find_draw_percentage_between_teams(team_1_id, team_2_id):
    total_matches = match_data.loc[(match_data['home_team_api_id'] == team_1_id) & (match_data['away_team_api_id'] == team_2_id)].shape[0]
    total_matches += match_data.loc[(match_data['home_team_api_id'] == team_2_id) & (match_data['away_team_api_id'] == team_1_id)].shape[0]
    
    if total_matches == 0:
        return 0
    
    draws = match_data.loc[(match_data['label'] == 1) & (match_data['home_team_api_id'] == team_1_id) & (match_data['away_team_api_id'] == team_2_id)].shape[0]
    draws += match_data.loc[(match_data['label'] == 1) & (match_data['home_team_api_id'] == team_2_id) & (match_data['away_team_api_id'] == team_1_id)].shape[0]
    
    draw_percentage = draws/total_matches
    return draw_percentage
    
match_data['draw_percentage'] = match_data.apply(lambda row: find_draw_percentage_between_teams(row.home_team_api_id, row.away_team_api_id), axis = 1)

### Remove unnecessary data

In [9]:
# shouldn't know these columns 
# TODO there are probably many more useless columns
columns_to_drop = ["match_api_id", "home_team_goal", "away_team_goal", "goal",
                   "shoton", "shotoff", "foulcommit", "card", "cross", "corner",
                   "possession", "date"]
match_data = match_data.drop(columns_to_drop, axis=1)


# drop home_player_X/Y and away_player_X/Y columns, not sure what they represent
for no in range(1,12):
    for char in ["X", "Y"]:
        match_data = match_data.drop("home_player_"+char+str(no), axis=1)
        match_data = match_data.drop("away_player_"+char+str(no), axis=1)

### Join tables

In [10]:
# drop null values and duplicates from team_attributes, otherwise the dataset size will grow to 700k rows
team_attributes = team_attributes.dropna(subset=['team_api_id'])
team_attributes = team_attributes.drop_duplicates(subset=['team_api_id'])

match_data = match_data.dropna(subset=['home_team_api_id', 'away_team_api_id'])

# create 2 versions with home/away prefixes for joining purposes
team_attributes_home = team_attributes.add_prefix("home_")
team_attributes_away = team_attributes.add_prefix("away_")

# join match data with home and away team attributes

match_data = pd.merge(match_data, team_attributes_home, how="left", left_on="home_team_api_id", right_on="home_team_api_id")
match_data = pd.merge(match_data, team_attributes_away, how="left", left_on="away_team_api_id", right_on="away_team_api_id")



# join match data with home and away players' attributes
player_attributes = player_attributes.drop_duplicates(subset=['player_api_id'])
for i in range(1,12):
    
    home_player_attributes_i = player_attributes.add_prefix("home_p_" + str(i) + "_")
    away_player_attributes_i = player_attributes.add_prefix("away_p_" + str(i) + "_")
    
    match_data = pd.merge(match_data, home_player_attributes_i, left_on="home_player_" + str(i), right_on="home_p_" + str(i) + "_player_api_id")
    match_data = pd.merge(match_data, away_player_attributes_i, left_on="away_player_" + str(i), right_on="away_p_" + str(i) + "_player_api_id")
    
    # delete temp dataframe to conserve memory
    del(home_player_attributes_i)


In [38]:
match_data.loc[0][columns=["home_p_1_overall_rating", "home_p2_overall_rating"]]

SyntaxError: invalid syntax (<ipython-input-38-a299391502fc>, line 1)

### Select the columns to be used as features

In [91]:

features = match_data[['season',
                        'month',
                        'stage', 
                       
                        #'home_player_1',
                        #'home_player_2',
                        #'home_player_3',
                        #'home_player_4',
                        #'home_player_5',
                        #'home_player_6',
                        #'home_player_7',
                        #'home_player_8',
                        #'home_player_9',
                        #'home_player_10',
                        #'home_player_11',
                        #'away_player_1',
                        #'away_player_2',
                        #'away_player_3',
                        #'away_player_4',
                        #'away_player_5',
                        #'away_player_6',
                        #'away_player_7',
                        #'away_player_8',
                        #'away_player_9',
                        #'away_player_10',
                        #'away_player_11',
                       
                        #'home_p_1_overall_rating',
                        #'away_p_1_overall_rating',
                        #'home_p_2_overall_rating',
                        #'away_p_2_overall_rating',
                        #'home_p_3_overall_rating',
                        #'away_p_3_overall_rating',
                        #'home_p_4_overall_rating',
                        #'away_p_4_overall_rating',
                        #'home_p_5_overall_rating',
                        #'away_p_5_overall_rating',
                        #'home_p_6_overall_rating',
                        #'away_p_6_overall_rating',
                        #'home_p_7_overall_rating',
                        #'away_p_7_overall_rating',
                        #'home_p_8_overall_rating',
                        #'away_p_8_overall_rating',
                        #'home_p_9_overall_rating',
                        #'away_p_9_overall_rating',
                        #'home_p_10_overall_rating',
                        #'away_p_10_overall_rating',
                        #'home_p_11_overall_rating',
                        #'away_p_11_overall_rating',
                       
                        #'home_buildUpPlaySpeed',
                        #'home_buildUpPlayPassing',
                        #'home_chanceCreationPassing',
                        #'home_chanceCreationCrossing',
                        #'home_chanceCreationShooting',
                        'home_defencePressure',
                        'home_defenceAggression',
                        'home_defenceTeamWidth',
                        #'home_win_percentage',
                        
                       
                        #'away_buildUpPlaySpeed',
                        #'away_buildUpPlayPassing',
                        #'away_chanceCreationPassing',
                        #'away_chanceCreationCrossing',
                        #'away_chanceCreationShooting',
                        'away_defencePressure',
                        'away_defenceAggression',
                        'away_defenceTeamWidth',
                        #'away_win_percentage',
                        
                        'home_team_win_percentage_against_away',
                        'away_team_win_percentage_against_home',
                        'draw_percentage',
                       
                        'B365H',
                        'B365D',
                        'B365A',
                        'BWH',
                        'BWD',
                        'BWA',
                        'IWH',
                        'IWD',
                        'IWA',
                        'LBH',
                        'LBD',
                        'LBA',
                        'WHH',
                        'WHD',
                        'WHA',
                        'VCH',
                        'VCD',
                        'VCA',
                        'label']]

features.head()

,season,month,stage,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,home_team_win_percentage_against_away,...,LBH,LBD,LBA,WHH,WHD,WHA,VCH,VCD,VCA,label
0,2008,3,26,60.0,70.0,60.0,70.0,50.0,70.0,0.500000,...,4.0,3.4,1.72,4.33,3.30,1.70,4.35,3.3,1.75,2.0
1,2014,12,21,46.0,48.0,45.0,60.0,70.0,70.0,0.750000,...,6.5,4.2,1.44,6.00,4.33,1.44,7.50,4.6,1.45,0.0
2,2011,12,17,60.0,70.0,60.0,70.0,50.0,70.0,0.500000,...,5.0,3.5,1.57,5.50,3.60,1.57,6.00,4.0,1.53,2.0
3,2012,8,3,65.0,60.0,70.0,70.0,50.0,70.0,0.750000,...,3.5,3.4,1.91,3.80,3.50,1.83,3.80,3.5,1.95,0.0
4,2014,2,25,60.0,60.0,65.0,60.0,70.0,70.0,0.571429,...,3.1,3.5,2.10,2.88,3.50,2.20,3.20,3.7,2.15,2.0


### Write data to csv

In [92]:
# drop rows with nulls
features = features.dropna()
features.to_csv("datasets/data.csv")

In [93]:
features.shape

(19487, 31)